In [1]:
import os
from dotenv import load_dotenv

# LangChain 관련 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings # Ollama 임베딩 추가
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.document import Document
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import WikipediaQueryRun, TavilySearchResults
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# .env 파일에서 환경 변수 로드
load_dotenv()

# Groq API 키 로드
groq_api_key = os.getenv("OPENAI_API_KEY")

print("라이브러리 임포트 및 Groq API 키 설정 완료.")
if not groq_api_key:
    print("경고: .env 파일에서 OPENAI_API_KEY를 찾을 수 없습니다.")

라이브러리 임포트 및 Groq API 키 설정 완료.


In [2]:
import re
from langchain_community.docstore.document import Document
from langchain_community.vectorstores import FAISS

print("상세 메뉴 데이터 파일을 파싱하여 벡터 DB를 생성합니다...")

file_path = "../data/cafe_menu_data.txt"

with open(file_path, 'r', encoding='utf-8') as f:
    content = f.read()

items = re.split(r'\n(?=\d+\.\s)', content.strip())
cafe_menu_data = [item.strip() for item in items if item.strip()]

documents = [Document(page_content=item) for item in cafe_menu_data]

# --- 이 부분이 변경됩니다 ---
# Ollama 임베딩 모델 사용
print("Ollama 임베딩 모델(bge-m3:latest)을 사용합니다...")
embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
# --------------------------

# FAISS 벡터 스토어 생성 시 변경된 모델 변수 사용
vector_db = FAISS.from_documents(documents, embeddings_model)

retriever = vector_db.as_retriever()

print(f"'{file_path}' 파일에서 {len(documents)}개의 메뉴를 성공적으로 파싱하여 벡터 DB 생성을 완료했습니다.")

상세 메뉴 데이터 파일을 파싱하여 벡터 DB를 생성합니다...
Ollama 임베딩 모델(bge-m3:latest)을 사용합니다...


C:\Users\user\AppData\Local\Temp\ipykernel_15052\3580237333.py:20: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings_model = OllamaEmbeddings(model="bge-m3:latest")


'../data/cafe_menu_data.txt' 파일에서 10개의 메뉴를 성공적으로 파싱하여 벡터 DB 생성을 완료했습니다.


In [3]:
# langchain.tools.retriever에서 create_retriever_tool을 임포하도록 수정합니다.
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import Tool
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper

print("도구 정의 중...")

# 도구 1: 카페 메뉴 DB 검색 도구
db_search_cafe_tool = create_retriever_tool(
    retriever,
    "db_search_cafe_func",
    "우리 카페에서 판매하는 메뉴(음료, 디저트)에 대한 정보를 찾을 때 사용합니다. 가격, 재료, 메뉴 설명 등의 질문에 답할 수 있습니다."
)

# 도구 2: 위키피디아 검색 도구
wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=2000)
wiki_tool = Tool(
    name="wikipedia",
    func=wikipedia_api_wrapper.run,
    description="음식이나 음료의 역사, 유래, 문화적 배경 등 일반적인 지식이 필요할 때 사용합니다."
)

# 도구 3: 최신 정보 웹 검색 도구 (Tavily)
tavily_tool = TavilySearchResults(
    name="tavily_search_func",
    description="최신 트렌드, 뉴스, 이벤트 등 실시간 정보나 최신 정보가 필요할 때 사용하는 웹 검색 엔진입니다."
)

# 사용할 도구들을 리스트로 묶기
tools = [db_search_cafe_tool, wiki_tool, tavily_tool]

print("도구 정의 완료.")
print(f"정의된 도구: {[tool.name for tool in tools]}")

도구 정의 중...
도구 정의 완료.
정의된 도구: ['db_search_cafe_func', 'wikipedia', 'tavily_search_func']


C:\Users\user\AppData\Local\Temp\ipykernel_15052\2859691027.py:25: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(


In [4]:
print("요구사항을 모두 반영한 최종 프롬프트 템플릿 작성 중...")

# 사용자의 상세 요구사항과 ReAct 에이전트의 기술적 요구사항을 모두 결합한 최종 프롬프트
prompt_template_text = """
당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI 어시스턴트입니다.
사용자의 질문에 대해 아래의 지침과 예제를 참고하여, 주어진 도구를 사용해 단계적으로 답변을 생성해야 합니다.

---
### 도구 목록 및 설명
당신이 사용할 수 있는 도구 목록입니다:
{tools}

### 도구 사용 원칙:
1. 카페 메뉴 관련 질문 (`db_search_cafe_func` 사용): 가격, 재료, 설명 등 카페 메뉴에 대한 질문은 **반드시** 이 도구를 가장 먼저 사용해야 합니다.
2. 역사/문화/일반 지식 (`wikipedia` 사용): 음식, 음료, 인물, 사건 등의 역사, 유래, 문화적 배경 등 일반적인 지식이 필요할 때 사용합니다.
3. 최신 트렌드/뉴스 (`tavily_search_func` 사용): 최신 유행, 뉴스, 실시간 정보 등 현재 시점의 정보가 필요할 때 사용합니다.
4. 복합 질문: 여러 도구를 순차적으로 사용하여 정보를 종합합니다.

### 답변 생성 예제 (Few-shot Example)
이 예제는 당신이 따라야 할 생각과 행동의 흐름을 보여줍니다. 이와 똑같은 방식으로 생각하고 행동해야 합니다.

Question: 아메리카노 정보와 커피의 역사에 대해 알려주세요.
Thought: 사용자가 '아메리카노 정보'와 '커피의 역사' 두 가지를 질문했습니다. '아메리카노 정보'는 카페 메뉴 정보이므로 `db_search_cafe_func`를, '커피의 역사'는 일반 지식이므로 `wikipedia`를 사용해야 합니다.
Action: db_search_cafe_func
Action Input: 아메리카노
Observation: 아메리카노: 4,500원, 에티오피아 원두를 사용한 깔끔한 맛의 커피입니다.
Thought: 이제 커피의 역사에 대해 알아보기 위해 위키피디아를 검색해야 합니다.
Action: wikipedia
Action Input: 커피의 역사
Observation: Page: Coffee. Summary: 커피(Coffee)는 에티오피아의 고원 지대에서 발견된 것으로 알려져 있으며, 칼디라는 목동의 전설이 유명합니다. 이후 아라비아 반도로 전파되어... (중략)
Thought: 두 가지 정보를 모두 얻었습니다. 이제 이 정보들을 종합하여 최종 답변을 생성합니다.
Final Answer: 네, 아메리카노와 커피의 역사에 대해 알려드릴게요. 저희 카페의 아메리카노는 에티오피아 원두를 사용한 깔끔한 맛의 커피이며, 가격은 4,500원입니다. 일반적으로 커피는 에티오피아에서 처음 발견되었다고 알려져 있으며, 이후 아라비아 반도를 거쳐 전 세계로 퍼져나가 오늘날 많은 사람들이 즐기는 음료가 되었습니다.
---

이제 실제 답변을 생성합니다.

사용 가능한 도구 이름 목록: [{tool_names}]

아래의 `Question`에 답변을 시작하세요. `Thought`부터 당신의 생각을 작성하고, 위 예제와 같이 `Action`, `Action Input`, `Observation`을 반복하여 최종 답변(`Final Answer`)을 도출해야 합니다.

Question: {input}
{agent_scratchpad}
"""

prompt = ChatPromptTemplate.from_template(prompt_template_text)

print("프롬프트 템플릿 작성 완료.")

요구사항을 모두 반영한 최종 프롬프트 템플릿 작성 중...
프롬프트 템플릿 작성 완료.


In [5]:
print("에이전트 및 실행기 생성 중...")

# --- 이 부분이 변경됩니다 ---
# LLM 모델을 Groq API를 사용하도록 설정
print("LLM으로 Groq API (llama3-8b-8192)를 사용합니다...")
llm = ChatOpenAI(
    api_key=groq_api_key, # .env 파일에서 로드한 키 사용
    base_url="https://api.groq.com/openai/v1",
    model="llama3-8b-8192",  # 현재 Groq에서 사용 가능한 Llama3 모델
    temperature=0.3
)
# --------------------------

# ReAct 에이전트 생성
agent = create_react_agent(llm, tools, prompt)

# 에이전트 실행기(Executor) 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

print("에이전트 및 실행기 생성 완료.")

에이전트 및 실행기 생성 중...
LLM으로 Groq API (llama3-8b-8192)를 사용합니다...
에이전트 및 실행기 생성 완료.


In [ ]:
# 테스트할 복합 질문
complex_question = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."

print(f"질문: {complex_question}\n")

# 에이전트 실행
# verbose=True로 인해 에이전트의 생각 과정이 모두 출력됩니다.
response = agent_executor.invoke({
    "input": complex_question
})

print("\n" + "="*50)
print(" 최종 답변 ".center(50, "="))
print("="*50)
print(response['output'])
print("="*50)

질문: 카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요.



> Entering new AgentExecutor chain...
Thought: 사용자가 '카페라떼와 어울리는 디저트'와 '라떼의 유래' 두 가지를 질문했습니다. '카페라떼와 어울리는 디저트'는 카페 메뉴 정보이므로 `db_search_cafe_func`를, '라떼의 유래'는 일반 지식이므로 `wikipedia`를 사용해야 합니다.

Action: db_search_cafe_func
Action Input: 카페라떼와 어울리는 디저트4. 바닐라 라떼
   • 가격: ₩6,000
   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽
   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.

2. 카페라떼
   • 가격: ₩5,500
   • 주요 원료: 에스프레소, 스팀 밀크
   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.

5. 카라멜 마키아토
   • 가격: ₩6,500
   • 주요 원료: 에스프레소, 스팀 밀크, 카라멜 시럽, 휘핑크림
   • 설명: 스팀 밀크 위에 에스프레소를 부어 만든 후 카라멜 시럽과 휘핑크림으로 마무리한 달콤한 커피입니다. 카라멜의 진한 단맛과 커피의 깊은 맛이 조화를 이루며, 시각적으로도 아름다운 층을 형성합니다.

8. 녹차 라떼
   • 가격: ₩5,800
   • 주요 원료: 말차 파우더, 스팀 밀크, 설탕
   • 설명: 고급 말차 파우더와 부드러운 스팀 밀크로 만든 건강한 음료입니다. 녹차의 은은한 쓴맛과 우유의 부드러움이 조화를 이루며, 항산화 성분이 풍부합니다. 달콤함 조절이 가능합니다.Thought: 사용자가 '카페라떼와